# Load and Examine the data

In [1]:
import pandas
print(pandas.__version__)

1.2.4


In [1]:
data_bucket = 'just-abdul-aws' #Definfing bucket name
subfolder = 'chap3' #Defining sub-folder name
dataset = 'churn_data.csv'

# Importing the necessary libraries

In [2]:
import pandas as pd
from time import sleep
import numpy as np
import boto3
import sagemaker
import s3fs
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics

In [6]:
role = sagemaker.get_execution_role()
sess = sagemaker.Session()
s3 = s3fs.S3FileSystem(anon=False)

In [17]:
df = pd.read_csv(f's3://{data_bucket}/{subfolder}/{dataset}')
df.head()

,churned,id,customer_code,co_name,total_spend,week_minus_4,week_minus_3,week_minus_2,last_week,4-3_delta,3-2_delta,2-1_delta
0,0,1,1826,Hoffman Martinez and Chandler,68567.34,0.81,0.02,0.74,1.45,-0.79,0.72,0.71
1,0,2,772,Lee Martin and Escobar,74335.27,1.87,1.02,1.29,1.19,-0.85,0.27,-0.10
2,0,3,479,Hobbs Mcdaniel and Baker,48746.22,1.21,0.70,1.04,2.12,-0.51,0.34,1.08
3,0,4,1692,Williams-Harris,64416.70,0.75,2.08,2.40,2.02,1.33,0.32,-0.38
4,0,5,2578,Beck-Snyder,71623.20,2.33,0.66,1.97,1.60,-1.67,1.31,-0.37


In [18]:
churned_or_not = df.churned.value_counts()
pd.DataFrame(churned_or_not)

,churned
0,2833
1,166


2833 stayed 
166 churned

In [ ]:
# Processing the Categorical Data

In [19]:
columns = df.columns.tolist()

Rather than encoding the CAT data, we'd remove them. The CAT data are unique to each customers i.e it represents each customers  and thus it won't improve the model in anyway.

In [20]:
encoded_data = df.drop(['customer_code', 'co_name'], axis=1)

In [21]:
encoded_data.head()

,churned,id,total_spend,week_minus_4,week_minus_3,week_minus_2,last_week,4-3_delta,3-2_delta,2-1_delta
0,0,1,68567.34,0.81,0.02,0.74,1.45,-0.79,0.72,0.71
1,0,2,74335.27,1.87,1.02,1.29,1.19,-0.85,0.27,-0.10
2,0,3,48746.22,1.21,0.70,1.04,2.12,-0.51,0.34,1.08
3,0,4,64416.70,0.75,2.08,2.40,2.02,1.33,0.32,-0.38
4,0,5,71623.20,2.33,0.66,1.97,1.60,-1.67,1.31,-0.37


# Create training, validation and test data sets

# Splitting Data

In [24]:
train_df, val_and_test_data = train_test_split(encoded_data, test_size=0.3,random_state=0)

val_df, test_df = train_test_split(val_and_test_data,test_size=0.3333,random_state=0)

In [25]:
train_data = train_df.drop(["id"], axis=1)
val_data = val_df.drop(["id"], axis=1)
test_data = test_df.drop(["churned"], axis=1) #Target Variable
test_data_NO_ID = test_data.drop(["id"], axis=1)

In [135]:
test_d = test_df.drop(["id"], axis=1)
test_d

,churned,total_spend,week_minus_4,week_minus_3,week_minus_2,last_week,4-3_delta,3-2_delta,2-1_delta
847,0,69813.37,0.13,0.85,1.75,0.40,0.72,0.90,-1.35
953,0,53273.07,0.92,0.31,0.21,2.01,-0.61,-0.10,1.80
2679,0,49347.67,1.46,0.35,1.61,1.80,-1.11,1.26,0.19
1767,0,13455.19,2.30,1.27,2.30,0.34,-1.03,1.03,-1.96
1244,0,4673.40,1.08,0.48,0.31,0.54,-0.60,-0.17,0.23
...,...,...,...,...,...,...,...,...,...
1949,0,41934.67,1.41,1.60,1.87,2.29,0.19,0.27,0.42
727,0,16296.03,0.35,0.56,1.34,0.50,0.21,0.78,-0.84
993,0,20127.28,0.55,1.38,2.02,0.33,0.83,0.64,-1.69
2402,0,50710.77,1.22,0.46,1.93,2.48,-0.76,1.47,0.55


In [126]:
test_data_NO_ID

,total_spend,week_minus_4,week_minus_3,week_minus_2,last_week,4-3_delta,3-2_delta,2-1_delta
847,69813.37,0.13,0.85,1.75,0.40,0.72,0.90,-1.35
953,53273.07,0.92,0.31,0.21,2.01,-0.61,-0.10,1.80
2679,49347.67,1.46,0.35,1.61,1.80,-1.11,1.26,0.19
1767,13455.19,2.30,1.27,2.30,0.34,-1.03,1.03,-1.96
1244,4673.40,1.08,0.48,0.31,0.54,-0.60,-0.17,0.23
...,...,...,...,...,...,...,...,...
1949,41934.67,1.41,1.60,1.87,2.29,0.19,0.27,0.42
727,16296.03,0.35,0.56,1.34,0.50,0.21,0.78,-0.84
993,20127.28,0.55,1.38,2.02,0.33,0.83,0.64,-1.69
2402,50710.77,1.22,0.46,1.93,2.48,-0.76,1.47,0.55


In [125]:
train_data

,churned,total_spend,week_minus_4,week_minus_3,week_minus_2,last_week,4-3_delta,3-2_delta,2-1_delta
2531,0,4580.79,1.69,1.26,0.86,0.53,-0.43,-0.40,-0.33
2439,0,27742.27,1.58,1.82,0.48,1.88,0.24,-1.34,1.40
1225,1,5643.40,0.82,1.98,0.12,0.63,1.16,-1.86,0.51
2966,0,65318.46,1.30,1.24,0.48,0.16,-0.06,-0.76,-0.32
1070,0,77398.39,2.39,2.21,2.17,2.25,-0.18,-0.04,0.08
...,...,...,...,...,...,...,...,...,...
763,0,71140.32,2.47,0.81,0.79,2.36,-1.66,-0.02,1.57
835,0,18235.49,0.43,0.28,0.77,2.19,-0.15,0.49,1.42
1653,0,26357.06,1.12,1.76,2.47,1.28,0.64,0.71,-1.19
2607,0,36296.59,1.46,0.97,1.41,0.64,-0.49,0.44,-0.77


# Converting Data to csv

In [26]:
train_file = train_data.to_csv(None, header=False, index=False).encode() # Doesn't include column header
val_file = val_data.to_csv(None, header=False, index=False).encode() #Encode is to ensure text in csv is saved in the right format
test_file = test_data.to_csv(None, header=False, index=False).encode()
test_file_NO_ID = test_data_NO_ID.to_csv(None, header=False, index=False).encode()

# Saving the CSV files to S3.

In [31]:
with s3.open(f'{data_bucket}/{subfolder}/processed/train.csv', 'wb') as f:
    f.write(train_file)
with s3.open(f'{data_bucket}/{subfolder}/processed/val.csv', 'wb') as f:
    f.write(val_file)
with s3.open(f'{data_bucket}/{subfolder}/processed/test.csv', 'wb') as f:
    f.write(test_file)
with s3.open(f'{data_bucket}/{subfolder}/processed/test_NO_ID.csv', 'wb') as f:
    f.write(test_file_NO_ID)

# Training the Model

In [42]:
%%time
from time import gmtime, strftime
from sagemaker.amazon.amazon_estimator import get_image_uri

job_name = "xgb-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
output_location = "s3://{}/{}/output/{}".format(data_bucket, subfolder, job_name)
image = sagemaker.image_uris.retrieve("xgboost", boto3.Session().region_name, "1")

sm_estimator = sagemaker.estimator.Estimator(
    image,
    role,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    input_mode="File",
    output_path=output_location,
    sagemaker_session=sess,
                                             )
sm_estimator.set_hyperparameters(
                        max_depth=3,
                        subsample=0.7,
                        objective='binary:logistic',
                        eval_metric='auc',
                        num_round=100,
                        early_stopping_rounds=10,
                        scale_pos_weight=17
                                )

train_file = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/processed/train.csv".format(data_bucket, subfolder),
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)
validation_file = sagemaker.inputs.TrainingInput(
    "s3://{}/{}/processed/val.csv".format(data_bucket, subfolder),
    distribution="FullyReplicated",
    content_type="text/csv",
    s3_data_type="S3Prefix",
)
data_channels = {"train": train_file, "validation": validation_file}

# Start training by calling the fit method in the estimator
sm_estimator.fit(inputs=data_channels, logs=True)

2021-06-11 07:31:53 Starting - Starting the training job...
2021-06-11 07:31:55 Starting - Launching requested ML instancesProfilerReport-1623396713: InProgress
......
2021-06-11 07:33:22 Starting - Preparing the instances for training.........
2021-06-11 07:34:48 Downloading - Downloading input data
2021-06-11 07:34:48 Training - Downloading the training image...
2021-06-11 07:35:23 Uploading - Uploading generated training model
2021-06-11 07:35:23 Completed - Training job completed
Arguments: train
[2021-06-11:07:35:10:INFO] Running standalone xgboost training.
[2021-06-11:07:35:10:INFO] File size need to be processed in the node: 0.12mb. Available memory size in the node: 8389.19mb
[2021-06-11:07:35:10:INFO] Determined delimiter of CSV input is ','
[07:35:10] S3DistributionType set as FullyReplicated
[07:35:10] 2099x8 matrix with 16792 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2021-06-11:07:35:10:INFO] Determined delimiter of CSV input is ',

# Batch Transform 

In [60]:
test_NO_ID = "test_NO_ID.csv"

In [ ]:
%%time

sm_transformer = sm_estimator.transformer(instance_count=1, instance_type="ml.m4.xlarge")

# start a transform job

input_location = "s3://just-abdul-aws/chap3/processed/test_NO_ID.csv"   # use input testdata without ID column

sm_transformer.transform(input_location, content_type="text/csv", split_type="Line")
sm_transformer.wait()

In [61]:
import json
import io
from urllib.parse import urlparse


def get_csv_output_from_s3(s3uri, file_name):
    parsed_url = urlparse(s3uri)
    bucket_name = parsed_url.netloc
    prefix = parsed_url.path[1:]
    s3 = boto3.resource("s3")
    obj = s3.Object(bucket_name, "{}/{}".format(prefix, file_name))
    return obj.get()["Body"].read().decode("utf-8")

# Prediction

In [62]:
output = get_csv_output_from_s3(sm_transformer.output_path, "{}.out".format(test_NO_ID))
output_df = pd.read_csv(io.StringIO(output), sep=",", header=None)
output_df.head(8)

,0
0,0.257038
1,0.257038
2,0.257038
3,0.257038
4,0.257038
5,0.257038
6,0.257038
7,0.718844


In [64]:
test = "test.csv"

In [ ]:
# content_type / accept and split_type / assemble_with are required to use IO joining feature
sm_transformer.assemble_with = "Line"
sm_transformer.accept = "text/csv"


input_location = "s3://{}/{}/processed/{}".format(data_bucket, subfolder, test)  # use input testdata without ID column


# start another transform job
sm_transformer.transform(
    input_location,
    split_type="Line",
    content_type="text/csv",
    input_filter="$[1:]",
    join_source="Input",
    output_filter="$[0,-1]",
                        )
sm_transformer.wait()


In [66]:
output = get_csv_output_from_s3(sm_transformer.output_path, "{}.out".format(test))
output_df = pd.read_csv(io.StringIO(output), sep=",", header=None)
output_df.head(8)

,0,1
0,848,0.257038
1,954,0.257038
2,2680,0.257038
3,1768,0.257038
4,1245,0.257038
5,688,0.257038
6,2448,0.257038
7,2993,0.718844


# Hosting 

checking if endpoint name exists and deleting if yes

In [119]:
endpoint_name = 'customer-churn'

try:
    sess.delete_endpoint(endpoint_name)
    print('Warning: Existing endpoint deleted to make way for your new endpoint.')
    sleep(30)
except:
    pass

In [88]:
from sagemaker.deserializers import JSONDeserializer
from sagemaker.serializers import CSVSerializer

In [120]:
predictor = sm_estimator.deploy(
                initial_instance_count=1,
                serializer=CSVSerializer(),
                instance_type='ml.m4.xlarge', 
                endpoint_name=endpoint_name)

-------------!

In [ ]:
#Test the Model

In [121]:
def predict(data, rows=300):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ""
    for array in split_array:
        predictions = ",".join([predictions, predictor.predict(array).decode("utf-8")])

    return np.fromstring(predictions[1:], sep=",")

In [136]:
predictions = predict(test_d.to_numpy()[:, 1:])

In [137]:
print(predictions)

[0.25703764 0.25703764 0.25703764 0.25703764 0.25703764 0.25703764
 0.25703764 0.71884364 0.25703764 0.25703764 0.25703764 0.25703764
 0.28889874 0.25703764 0.25703764 0.25703764 0.25703764 0.25703764
 0.25703764 0.25703764 0.25703764 0.25703764 0.32528025 0.4729439
 0.25703764 0.25703764 0.25703764 0.25703764 0.25703764 0.71385962
 0.25703764 0.25703764 0.25703764 0.25703764 0.25703764 0.25703764
 0.25703764 0.25703764 0.71884364 0.25703764 0.25703764 0.25703764
 0.25703764 0.25703764 0.25703764 0.25703764 0.25703764 0.25703764
 0.25703764 0.25703764 0.32528025 0.25703764 0.25703764 0.25703764
 0.25703764 0.25703764 0.25703764 0.25703764 0.25703764 0.32528025
 0.25703764 0.25703764 0.25703764 0.25703764 0.25703764 0.25703764
 0.25703764 0.25703764 0.71884364 0.25703764 0.47277451 0.25703764
 0.25703764 0.71884364 0.25703764 0.25703764 0.25703764 0.25703764
 0.25703764 0.25703764 0.25703764 0.25703764 0.25703764 0.25703764
 0.25703764 0.25703764 0.25703764 0.25703764 0.25703764 0.25703

In [138]:
pd.crosstab(
    index=test_df.iloc[:, 0],
    columns=np.round(predictions),
    rownames=["actual"],
    colnames=["predictions"],
)

predictions,0.0,1.0
actual,,
0,264,23
1,1,12


In [139]:
test_data = test_d

In [140]:
pred = np.round(predictions)
pred = pd.DataFrame(pred)
pred.columns=['Prediction']

In [141]:
test_data = test_data.reset_index(drop=True)

In [142]:
result = pd.merge(pred,test_data, left_index=True, right_index=True)

In [143]:
result = result.astype('int32')

In [144]:
result

,Prediction,churned,total_spend,week_minus_4,week_minus_3,week_minus_2,last_week,4-3_delta,3-2_delta,2-1_delta
0,0,0,69813,0,0,1,0,0,0,-1
1,0,0,53273,0,0,0,2,0,0,1
2,0,0,49347,1,0,1,1,-1,1,0
3,0,0,13455,2,1,2,0,-1,1,-1
4,0,0,4673,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
295,0,0,41934,1,1,1,2,0,0,0
296,0,0,16296,0,0,1,0,0,0,0
297,0,0,20127,0,1,2,0,0,0,-1
298,0,0,50710,1,0,1,2,0,1,0


In [145]:
from sklearn.metrics import accuracy_score

In [146]:
print("Baseline Accuracy = {}".format(1- np.unique(df['churned'], return_counts=True)[1][1]/(len(df['churned']))))
print("Accuracy Score = {}".format(accuracy_score(test_d['churned'], result['Prediction'])))

Baseline Accuracy = 0.944648216072024
Accuracy Score = 0.92


# Clean up

In [ ]:
sess.delete_endpoint(endpoint_name)